In [4]:
from pretty_midi import PrettyMIDI, instrument_name_to_program, Instrument, note_name_to_number, Note
from os import system
import pandas as pd

In [31]:
def synthesize(midi_file, chords, beats, durations, output, instrument="String Ensemble 1", velocity=50, group=3, melody_only=True, to_mp3=False):
    KEYS = ("C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B")
    midi_data = PrettyMIDI(midi_file)
    if melody_only:
        midi_data.instruments = [midi_data.instruments[0]]
    program = instrument_name_to_program(instrument)
    accompany = Instrument(program=program, name="accompany")
    old_chord = []
    for chord, beat, duration in zip(chords + [[]], beats + [0], durations + [0]):
        if chord != old_chord:
            if old_chord:
                for bit, key in zip(old_chord, KEYS):
                    if bit:
                        note_number = note_name_to_number(f"{key}{group}")
                        note = Note(velocity=velocity, pitch=note_number, start=old_beat, end=old_beat+total_duration)
                        accompany.notes.append(note)
            old_beat = beat
            total_duration = duration
            old_chord = chord
        else:
            total_duration += duration
    midi_data.instruments.append(accompany)
    midi_data.write(output)
    if to_mp3:
        system(f"Musescore4 {output} -o {output.replace('mid', 'mp3')}")

In [25]:
def parse_csv(csv_file):
    df = pd.read_csv(csv_file)
    chords = df["chord"].apply(lambda x: eval(x.replace(".", ","))).to_list()
    beats = df["time"].to_list()
    durations = df["duration"].to_list()
    return chords, beats, durations

In [32]:
synthesize(
    r"POP909\POP909\001\001.mid", 
    *parse_csv(r"POP909\POP909\001\melody_chord_1_2_beat.csv"),
    "test.mid"
)